<a href="https://colab.research.google.com/github/lalasray/Render-3D-pose-from-a-video/blob/main/Notebook/Facebook_VideoPose3D_Inference_in_the_wild_Detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install pytorch & Caffe 
!wget https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!tar xvjf pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

In [ ]:
#Install coco dataset
!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

In [ ]:
#Install facebook detectron
!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make
!python detectron/detectron/tests/test_spatial_narrow_as_op.py

In [ ]:
#copy file from VideoPose3d
!git clone https://github.com/facebookresearch/VideoPose3D
!cp VideoPose3D/inference/infer_video.py detectron/tools/infer_video.py

In [ ]:
#copy file from helper_video2bvh
!git clone https://github.com/lalasray/helper_video2bvh
!mv "/content/helper_video2bvh/bvh_skeleton" "/content"
!mv "/content/helper_video2bvh/utils" "/content"
!mv "/content/helper_video2bvh/cameras.h5" "/content"

In [ ]:
#Download Pretrained Human3.6m Coco Model
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

In [ ]:
!gdown --id 1FSztcynGULFT7nkx6UIppIBy0dBwnmet

In [ ]:
#Download Youtube Video for 3D Pose Estimation
YOUTUBE_ID ='7YY1od0aRxc'


!pip install -q youtube-dl
#download video
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

!mkdir videos   
  
# cut the first 9 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:00 -t 00:00:05 videos/video.mp4

#!ffmpeg -i videos/video.mp4 -filter "minterpolate='fps=50'" -crf 0 videos/video50fps.mp4

In [ ]:
#Compute 2D Coordinates with Detectron
!mkdir output
!python detectron/tools/infer_video.py \
    --cfg detectron/configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
    --output-dir output \
    --image-ext mp4 \
    --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   videos 

In [ ]:
# fit detectron output to videopose input
!mkdir ./VideoPose3D/data/detectronoutput
!cp output/video.mp4.npz VideoPose3D/data/detectronoutput/video.mp4.npz
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectronoutput -o myvideos
os.chdir('../../')

In [11]:
#Compute 3D Joints with VideoPose3D
!cp ./videos/video.mp4 VideoPose3D/video.mp4
os.chdir('VideoPose3D')
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_243_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_243_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='outputfile', viz_limit=-1, viz_no_ground_truth=False, viz_output='output.mp4', viz_size=6, viz_skip=0, viz_subject='video.mp4', viz_video='video.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO

In [12]:
#inspect joints export 

import numpy as np
data  = np.load('outputfile.npy')
lst = data
for item in lst:
    print(item)
 


[[-0.05967337  0.01900988  0.6054319 ]
 [-0.21766105  0.03475317  0.5716969 ]
 [-0.38878965  0.49011362  0.6427333 ]
 [-0.51553327  0.91369504  0.78883976]
 [ 0.09831279  0.00327156  0.6398731 ]
 [ 0.39435944  0.42515707  0.7172415 ]
 [ 0.40618658  0.8279179   0.9361527 ]
 [-0.09183919 -0.24730887  0.55617464]
 [-0.12847775 -0.48829347  0.43033457]
 [-0.136931   -0.5663342   0.3245926 ]
 [-0.14916232 -0.6890182   0.35046458]
 [ 0.03675654 -0.44720826  0.46269223]
 [ 0.21710977 -0.19332272  0.5176925 ]
 [ 0.05310699 -0.02947938  0.3003055 ]
 [-0.273409   -0.3922491   0.44189113]
 [-0.38731527 -0.08371571  0.47773775]
 [-0.2352505  -0.02687862  0.24737477]]
[[-4.26595807e-02  1.61324684e-02  5.95486939e-01]
 [-2.00904369e-01  3.15668508e-02  5.66374540e-01]
 [-3.71177077e-01  4.89460230e-01  6.55542195e-01]
 [-4.92824525e-01  9.06908691e-01  8.06526482e-01]
 [ 1.15583494e-01  7.00892881e-04  6.25324965e-01]
 [ 4.09488946e-01  4.35225934e-01  7.07845926e-01]
 [ 4.58136350e-01  8.43996167e

In [13]:
#display video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

In [14]:
#Download Joint Export & Video
from google.colab import files


files.download('output.mp4')
files.download('outputfile.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import h5py
from utils import smooth, vis, camera
subject = 'S1'
cam_id = '55011271'
cam_params = camera.load_camera_params('/content/cameras.h5')[subject][cam_id]
R = cam_params['R']
T = 0
azimuth = cam_params['azimuth']

pose3d_world = camera.camera2world(pose=data, R=R, T=T)
pose3d_world[:, :, 2] -= np.min(pose3d_world[:, :, 2]) # rebase the height

pose3d_file =  'outputfile.npy'
np.save(pose3d_file, pose3d_world)

In [16]:
from pathlib import Path
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

video_file = Path('/content/VideoPose3D/output.mp4') # video file to process
output_dir = Path(f'miscs/{video_file.stem}_cache')
if not output_dir.exists():
    os.makedirs(output_dir)

bvh_file = output_dir / f'{video_file.stem}.bvh'
cmu_skel = cmu_skeleton.CMUSkeleton()
channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)

In [17]:
output = 'miscs/h36m_cxk.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()
_ = h36m_skel.poses2bvh(pose3d_world, output_file=output)

files.download(output)
files.download(bvh_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>